In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import rioxarray

In [ ]:
url = "https://github.com/opengeos/datasets/releases/download/raster/LC09_039035_20240708_90m.tif"
data = rioxarray.open_rasterio(url)
data

In [ ]:
data.dims

In [ ]:
data.coords

In [ ]:
print(data.attrs)

In [ ]:
data.rio.crs

In [ ]:
data.rio.transform()

In [ ]:
data.rio.write_crs("EPSG:32611" , inplace =True)

In [ ]:
data_reproject = data.rio.reproject("EPSG:4326")
print(data_reproject.rio.crs)

In [ ]:
bbox = [-115.391, 35.982, -114.988, 36.425]

clipped_data = data_reproject.rio.clip_box(*bbox)

In [ ]:
clipped_data.shape

In [ ]:
import geopandas as gpd

geojson_path = "https://github.com/opengeos/datasets/releases/download/places/las_vegas_bounds_utm.geojson"
bounds =gpd.read_file(geojson_path)

clipped_data2=data.rio.clip(bounds.geometry, bounds.crs)


In [ ]:
clipped_data2.shape

In [ ]:
resampled_data = data.rio.reproject(data.rio.crs, resolution=(1000,1000))

In [ ]:
resampled_data.shape

In [ ]:
min_x, max_x = -115.391,-114.988
min_y, max_y = 35.982,36.425

subset = data_reproject.sel(
    x=slice(min_x, max_x), y=slice(min_y, max_y), band=[3,2,1]
)

subset.shape

In [ ]:
plt.figure(figsize=(8,8))
data_reproject.sel(band=[4,3,2]).plot.imshow(vmin=0, vmax=0.3)
plt.title("Landsat Image Covering Las Vegas")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [ ]:
fig, ax =plt.subplots(figsize=(8,8))
data.attrs["long_name"] = "Surface Reflectance"
data.sel(band=4).plot.imshow(ax=ax, vmin=0, vmax=0.4, cmap="gray")
bounds.boundary.plot(ax=ax, color="red")
plt.title("Raster with Vector Overlay")
plt.show()

In [ ]:
data.rio.to_raster("output_raster.tif")